# Ad-Hoc RHWMD Fun

In [54]:
import pickle

from tabulate import tabulate

from ungol.wmd import wmd
from ungol.wmd import sim
from ungol.wmd import rhwmd

In [2]:
with open('../opt/databases/180921_ungol.db.pickle', 'rb') as fd:
    db = pickle.load(fd)

print(str(db))

TODO: loading config from ...ca/ungol-models/conf/logging.conf
recalculating avg doclen

VNGOL database
  containing: 294659 documents
  vocabulary: 400000 words
  filtering: 0 stopwords
  code size: 256 bits
  tokens: 258391
  avg. doc length: 197.35699
  skipped: 8



In [8]:
tokens1 = 'auf der mauer auf der lauer sitzt ne kleine wanze'.split(' ')
tokens2 = 'ein marienkäfer schläft auf dem zaun'.split(' ')
tokens3 = 'ich möchte diesen teppich nicht kaufen'.split(' ')

In [10]:
doc1 = wmd.Doc.from_tokens('wanze', tokens1, db.docref)
doc2 = wmd.Doc.from_tokens('marienkäfer', tokens2, db.docref)
doc3 = wmd.Doc.from_tokens('teppich', tokens3, db.docref)

In [41]:
def docstr(doc):
    str_buf = []
    header_knn = tuple('{}-nn'.format(k) for k in doc.ref.meta['knn'])
    header = ('word', 'tc', ) + header_knn

    bits = doc.ref.codemap.shape[1] * 8
    
    tab_data = []
    row_data = doc.tokens, doc.cnt, doc.codes, doc.dists
    
    for token, cnt, code, dist in zip(*row_data):
        dist = tuple('{:.3f}'.format(dist / bits) for dist in dist)
        tab_data.append((token, cnt, ) + dist)

    str_buf += [tabulate(tab_data, headers=header)]

    str_buf.append('')
    return '\n'.join(str_buf)

In [42]:
print(docstr(doc1))

word      tc    1-nn    11-nn    101-nn    1001-nn    3999-nn
------  ----  ------  -------  --------  ---------  ---------
auf        2   0.219    0.23      0.262      0.289      0.309
der        2   0.168    0.211     0.246      0.281      0.305
mauer      1   0.18     0.223     0.262      0.316      0.336
lauer      1   0.266    0.285     0.309      0.332      0.348
sitzt      1   0.211    0.262     0.281      0.312      0.332
ne         1   0.227    0.262     0.293      0.324      0.344
kleine     1   0.137    0.246     0.277      0.305      0.32
wanze      1   0.188    0.242     0.293      0.34       0.359



In [43]:
print(docstr(doc2))

word           tc    1-nn    11-nn    101-nn    1001-nn    3999-nn
-----------  ----  ------  -------  --------  ---------  ---------
ein             1   0.148    0.203     0.23       0.277      0.309
marienkäfer     1   0.156    0.188     0.266      0.32       0.348
schläft         1   0.188    0.23      0.266      0.309      0.336
auf             1   0.219    0.23      0.262      0.289      0.309
dem             1   0.176    0.219     0.25       0.285      0.309
zaun            1   0.199    0.246     0.297      0.328      0.344



In [44]:
print(docstr(doc3))

word       tc    1-nn    11-nn    101-nn    1001-nn    3999-nn
-------  ----  ------  -------  --------  ---------  ---------
ich         1   0.141    0.184     0.223      0.27       0.301
möchte      1   0.18     0.207     0.238      0.273      0.301
diesen      1   0.145    0.207     0.242      0.273      0.297
teppich     1   0.141    0.246     0.301      0.328      0.344
nicht       1   0.137    0.18      0.215      0.262      0.293
kaufen      1   0.133    0.188     0.258      0.305      0.332



In [47]:
db += doc1
db += doc2
db += doc3

AssertionError: "wanze" already indexed

In [65]:
def statprint(stats):
    for name, cols in stats.local_columns:
        print('\n', f"'{name}': (")
        col1, col2 = (repr(list(col)) for col in cols)
        print('  ', col1, ',')
        print('  ', col2, '),')

In [66]:
stats = sim.rhwmd(db, 'wanze', 'marienkäfer', verbose=True, strategy=rhwmd.Strategy.SUM)
print(str(stats))
statprint(stats)


rhwmd score : 1.4572827183329489

--------  ---
strategy  SUM
--------  ---



+----------------------------------------------------------------------------------------------+
|                                                                                              |
|  comparing: "wanze" to "marienkäfer"                                                         |
|                                                                                              |
|  -----  --------                                                                             |
|  score  0.711667                                                                             |
|  -----  --------                                                                             |
|                                                                                              |
|                                                                                              |
|    no  token    nn              sim    tf(tok

In [67]:
stats = sim.rhwmd(db, 'wanze', 'teppich', verbose=True, strategy=rhwmd.Strategy.SUM)
print(str(stats))
statprint(stats)


rhwmd score : 1.317160596374639

--------  ---
strategy  SUM
--------  ---



+-----------------------------------------------------------------------------------------+
|                                                                                         |
|  comparing: "wanze" to "teppich"                                                        |
|                                                                                         |
|  -----  --------                                                                        |
|  score  0.642874                                                                        |
|  -----  --------                                                                        |
|                                                                                         |
|                                                                                         |
|    no  token    nn         sim    tf(token)    idf(token)    idf(nn)     idf    weight  |
|